# Agent

## Review 
우리는 라우터를 구축했습니다.
- 우리의 채팅 모델은 사용자 입력에 따라 도구를 호출할지 말지를 결정합니다.
- 조건부 엣지를 사용하여 도구를 호출할 노드로 라우팅하거나 단순히 종료합니다.

![Screenshot 2024-08-21 at 12.44.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0ba0bd34b541c448cc_agent1.png)


## Goals
이제 이를 일반적인 에이전트 아키텍처로 확장할 수 있습니다.

위의 라우터에서는 모델을 호출하고, 모델이 도구 호출을 선택하면 사용자에게 ToolMessage를 반환했습니다.

하지만, 만약 그 ToolMessage를 단순히 모델에게 다시 전달한다면 어떨까요?

모델이 (1) 다른 도구를 호출하거나 (2) 직접 응답하도록 할 수 있습니다.

이것이 ReAct라는 일반 에이전트 아키텍처의 직관입니다.

* `act` - 모델이 특정 도구를 호출하도록 합니다.
* `observe` - 도구의 출력을 모델에게 다시 전달합니다.
* `reason` - 모델이 도구 출력에 대해 이유를 분석하여 다음에 무엇을 할지 결정하도록 합니다 (예: 다른 도구를 호출하거나 직접 응답).
이와 같은 [범용 아키텍처](https://blog.langchain.dev/planning-for-agents/)는 다양한 종류의 도구에 적용될 수 있습니다.

![Screenshot 2024-08-21 at 12.45.43 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0b4a2c1e5e02f3e78b_agent2.png)

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph

In [1]:
import os, getpass

def _set_env(var: str):
    
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    print(f"Multiplying {a} and {b}")
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    print(f"Adding {a} and {b}")
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    print(f"Dividing {a} by {b}")
    return a / b

tools = [add, multiply, divide]
llm = ChatOpenAI(model="gpt-4o")

# 이 ipynb에서는 수학 연산이 일반적으로 순차적으로 수행되기 때문에 parallel tool calling을 false로 설정했습니다.
# 이번에는 수학 연산을 수행할 수 있는 3개의 도구가 있습니다.
# 참고로, OpenAI 모델은 효율성을 위해 기본적으로 병렬 도구 호출을 사용합니다. 자세한 내용은 https://python.langchain.com/docs/how_to/tool_calling_parallel/ 을 참고하세요.
# 다양한 방식으로 실험해 보시고 모델이 수학 방정식에 대해 어떻게 동작하는지 확인해보세요!
llm_with_tools = llm.bind_tools(tools)

In [3]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

이전과 같이, 우리는 `MessagesState`를 사용하고 도구 목록을 가진 Tools 노드를 정의합니다.

`Assistant` 노드는 바인딩된 도구와 함께 작동하는 모델입니다.

`Assistant`와 `Tools` 노드로 구성된 그래프를 생성합니다.

그리고 `tools_condition` 엣지를 추가하여, `Assistant`가 도구를 호출하는지에 따라 `End` 또는 `Tools`로 라우팅합니다.

이제 한 가지 새로운 단계를 추가합니다:

`Tools` 노드를 `Assistant`에 다시 연결하여 루프를 형성합니다.

`Assistant` 노드가 실행된 후, `tools_condition`은 모델의 출력이 도구 호출인지 확인합니다.
만약 도구 호출이면, 흐름은 `Tools` 노드로 이동합니다.
그리고 `Tools` 노드는 다시 `Assistant`에 연결됩니다.

이 루프는 모델이 계속 도구를 호출하기로 결정하는 한 반복됩니다.
만약 모델의 응답이 도구 호출이 아니라면, 흐름은 END로 이동하여 프로세스를 종료합니다.

In [4]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

ImportError: cannot import name 'tools_condition' from 'langgraph.prebuilt' (unknown location)

In [ ]:
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = react_graph.invoke({"messages": messages})

In [ ]:
for m in messages['messages']:
    m.pretty_print()